In [40]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
import re

In [123]:
article = open("nytimes article.txt", "r")

sentences = []
for line in article:
    line = line.rstrip()
    line = line.split(".")
    sentences.append(line)
sentences[0]

['WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria',
 '']

In [124]:
sentences = [y for x in sentences for y in x] # flatten list

In [125]:
print(len(sentences))

84


In [126]:
sentences = [s for s in sentences if len(s) != 0]

In [127]:
print(len(sentences))

55


In [128]:
from nltk.tokenize import sent_tokenize
new_sentences = []
for s in sentences:
  new_sentences.append(sent_tokenize(s))

new_sentences = [y for x in new_sentences for y in x] # flatten list

In [129]:
new_sentences[:5]

['WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria',
 'Mr',
 ' Trump announced Mr',
 ' Mattis’s resignation in two tweets Thursday evening, and said the retired four-star Marine general turned Pentagon chief will leave at the end of February',
 'Officials said Mr']

In [130]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [131]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(new_sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [132]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [133]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [134]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [135]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [136]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [137]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [138]:
for i in range(len(sentence_vectors)):
    sentence_vectors[i] = sentence_vectors[i].reshape(1, 100)

In [139]:
print(len(sentence_vectors))

55


In [140]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [141]:
print(sim_mat.shape)

(55, 55)


In [142]:
from sklearn.metrics.pairwise import cosine_similarity


In [143]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i], sentence_vectors[j])[0,0]

In [144]:
import networkx as nx

nx_graph = nx.from_numpy_matrix(sim_mat)
scores = nx.pagerank(nx_graph)

In [145]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [146]:
for i in range(5):
    print(ranked_sentences[i][1], end="\n\n")

WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria

 The president said he would name Mr

 Mattis had told close friends that he would continue in the job despite his deteriorating relationship with Mr

Over the past six months, the president and the defense chief have also found themselves at odds over NATO policy, whether to resume large-scale military exercises with South Korea and, privately, whether Mr

The president’s tweets announcing the departure of his defense secretary shocked officials at the Pentagon, who as recently as Thursday afternoon were insisting that Mr

